In [ ]:
from sklearn.linear_model import LogisticRegressionCV
import pandas as pd
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
from datetime import date


%cd "C:\Users\artemiyburov\prog\"

In [ ]:
df = pd.read_csv("covid-19\data\worldwide-aggregated.csv")
#df.plot(y='Confirmed', use_index = True)

x = df.index       #features
y = df['Confirmed']#labels
x_predicted = np.arange(150)

In [ ]:
def logistic_function(x,A,x0,k,off):
    return A / (1 + np.exp(-k*(x-x0)))+off

In [ ]:
popt, pcov = curve_fit(logistic_function, x, y)
plt.scatter(x,y,label='Data')
plt.plot(x_predicted, logistic_function(x_predicted, *popt), 'r-',label='Fitted function')
plt.legend()

In [ ]:
#create a valid filename from string
import unicodedata
import string

valid_filename_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
char_limit = 255

def clean_filename(filename, whitelist=valid_filename_chars, replace=' '):
    # replace spaces
    for r in replace:
        filename = filename.replace(r,'_')
    
    # keep only valid ascii chars
    cleaned_filename = unicodedata.normalize('NFKD', filename).encode('ASCII', 'ignore').decode()
    
    # keep only whitelisted chars
    cleaned_filename = ''.join(c for c in cleaned_filename if c in whitelist)
    if len(cleaned_filename)>char_limit:
        print("Warning, filename truncated because it was over {}. Filenames may no longer be unique".format(char_limit))
    return cleaned_filename[:char_limit]

In [ ]:
df_by_country = pd.read_csv("covid-19\data\countries-aggregated.csv")
#df.loc[df_by_country["Country"] == "South Africa"]
for country in df_by_country.Country.unique():
    #if country == 'United Kingdom':#replace country here or add or conditions for comparison
    df_country = df_by_country.loc[df_by_country["Country"] == country].reset_index()
    x = df_country.index       #features
    y = df_country['Confirmed']#labels
    y_max = y.max()
    y_norm = y/y_max
    x_predicted = np.arange(200)
    try:
        popt, pcov = curve_fit(logistic_function, x, y_norm)
    except:
        continue
    y_predicted = y_max*logistic_function(x_predicted, *popt)
    sigma = np.sqrt(np.diagonal(pcov))#sqrt of covariance matrix
    y_predicted_sup = y_max*logistic_function(x_predicted, *(popt+2*sigma))#2 standard dev
    y_predicted_inf = y_max*logistic_function(x_predicted, *(popt-2*sigma))#2 standard dev
    thickness = max(abs(y_predicted_sup-y_predicted_inf))
    if thickness>.1 and thickness<y_max and y_max<=y_predicted.max():
        #plot
        fig = plt.figure()
        plt.scatter(x,y,label='Data '+country)
        plt.plot(x_predicted, y_predicted, 'r-',label='Total cases fit '+country)
        plt.fill_between(x_predicted, y_predicted_inf, y_predicted_sup, color = 'black', alpha = 0.15)
        plt.legend()
        #save into a file
        today = date.today()
        fig.savefig("epidemic-datathon\plots\{}-total-case-prediction-{}".format((clean_filename(country)), today))